In [220]:
import pandas as pd
import naya
import json
from itertools import islice
from os import listdir
from os.path import isfile, join


In [221]:
fp = open('../raw_data/export_23-5_minified.json', 'r')
messages = naya.stream_array(naya.tokenize(fp))


In [199]:
keys_1 = ['city', 'client_event_time', 'country', 'device_model', 'event_id', 'event_type', 'is_attribution_event', 'language', 'os_version', 'region', 'session_id', 'start_version', 'user_creation_time', 'user_id']

keys_2 = ['user_properties']

keys_3 = ['event_properties']

14

**#1. Step 1 : testing for a short list of lines**




In [168]:
#extract x lines
sample = []
for i in range(133):
    temp = next(messages)
    sample.append(temp)

In [145]:
#prepare json file : a list of dicts
json_1 = []
for i in range(len(sample)):
    dico = {}
    for key in keys_1:
        dico[key] = sample[i][key]
        
    json_1.append(dico)

In [148]:
type(json_1)

list

In [147]:
#create json file
with open('../marbotic/data/event_test_random.json', 'w') as f:
    json.dump(json_1, f, indent=None)    


**#2. Step 2 : slicing json in 100k lines and iterate to upload to BQ**




In [230]:
my_list = []
for elem in islice(messages,4000,4024):    
    
    
    my_list.append(elem)

my_list



[{'city': 'Morton',
  'client_event_time': '2022-03-04 01:37:13.539000',
  'country': 'United States',
  'device_model': 'iPad13,2',
  'event_id': 8608,
  'event_type': '[Action] Click',
  'is_attribution_event': False,
  'language': 'English',
  'os_version': '15.1',
  'region': 'Illinois',
  'session_id': 1646357767279,
  'start_version': '1.0.5',
  'user_creation_time': '2021-12-25 14:42:58.237000',
  'user_id': '66657',
  'user_properties': {'Newsletter': 'True',
   'Products': ['Smart Letters'],
   'Pieces': ['Uppercase Letters'],
   'Language': 'English',
   'Type': 'Parent',
   'Country': 'US',
   'Games': ['Smart Shapes', 'Marbotic']},
  'event_properties': {'[Game] Code': 'Marbotic',
   '[Scene] Name': 'NavigationActivitySelectionLetters',
   '[Profile] Age (days)': 2299,
   '[Game] Language': 'en_US',
   '[Action] Element Type': 'Button',
   '[Scene] Section': 'ABC',
   '[Action] Element Name': 'Play New Activity'}},
 {'city': 'Morton',
  'client_event_time': '2022-03-04 01:3

In [217]:
slicing = []
for i in range(4,15):
    step = 100000
    if i < 14:
        slicing.append(i*step)
    else:
        slicing.append((i-1)*step+1)
       
slicing

[400000,
 500000,
 600000,
 700000,
 800000,
 900000,
 1000000,
 1100000,
 1200000,
 1300000,
 1300001]

In [209]:
#extract all lines :scream:
def create_json(messages,slicing,keys):
    #for each batch of x ines create a simili json (sample = list of x dicts)
    for index,j in enumerate(slicing):
        sample = []
        json_temp = []
        if index != len(slicing)-1:
            for elem in islice(messages,j,slicing[index+1]):  
                sample.append(elem)
        else:
            for elem in islice(messages,j,None):
                sample.append(elem)
    #select the columns we are interested in to describe an event
        for i in range(len(sample)):
            dico = {}
            for key in keys:
                dico[key] = sample[i][key]
            json_temp.append(dico)
    #transform the list of dicts in json file & save it locally
        with open(f'../marbotic/data/event_test_100k_{j}.json', 'w') as f:
            json.dump(json_temp, f, indent=None)
        
    #transform intro newline format json
        !cat $f'../marbotic/data/event_test_100k_{j}.json' | jq -c '.[]' > $f'../marbotic/data/event_test_100k_{j}_f.json'
        

In [218]:
create_json(messages,slicing,keys_1)

KeyboardInterrupt: 

In [ ]:
!ls ../marbotic/data


In [201]:
!bq load --autodetect --noreplace --source_format=NEWLINE_DELIMITED_JSON marbotic_dataset.events ../marbotic/data/event_test_100k_800000_f.json

Waiting on bqjob_r69db815d9fed2e56_0000018142a13a32_1 ... (0s) Current status: DONE   
Error in query string: Error processing job
'marbotic:bqjob_r69db815d9fed2e56_0000018142a13a32_1': Schema has no fields.
Table: events_c00fadcd_60dd_497d_b26c_7bf5f2426197_source


In [188]:
list_path = "../marbotic/data"


In [191]:
fichiers = [f for f in listdir(list_path) if isfile(join(list_path, f))]

In [198]:
for elmt in fichiers:
    name = join(list_path, elmt)
    print(name)
    !bq load --autodetect --noreplace --source_format=NEWLINE_DELIMITED_JSON marbotic_dataset.events "$name"

../marbotic/data/event_test_100k_600000_f.json
Waiting on bqjob_r2151c4a6492939ac_00000181428cda72_1 ... (0s) Current status: DONE   
Error in query string: Error processing job
'marbotic:bqjob_r2151c4a6492939ac_00000181428cda72_1': Schema has no fields.
Table: events_41139204_ccf0_4c86_8e53_9d3d203154bd_source
../marbotic/data/event_test_100k_800000_f.json
Waiting on bqjob_r5b0fff8af7a24100_00000181428ce81f_1 ... (0s) Current status: DONE   
Error in query string: Error processing job
'marbotic:bqjob_r5b0fff8af7a24100_00000181428ce81f_1': Schema has no fields.
Table: events_dc5ff1e1_3ec5_4c66_854f_c6c05e0acf9a_source
../marbotic/data/event_test_100k_900000_f.json
Waiting on bqjob_r513b9aacbd1c9b59_00000181428cf508_1 ... (0s) Current status: DONE   
Error in query string: Error processing job
'marbotic:bqjob_r513b9aacbd1c9b59_00000181428cf508_1': Schema has no fields.
Table: events_82cfa476_75b1_4e96_be6e_7b086453a06f_source
../marbotic/data/event_test_100k_400000_f.json
Waiting on bqj

In [95]:
!bq show marbotic_dataset.event_properties




Table marbotic:marbotic_dataset.event_properties

   Last modified                 Schema                Total Rows   Total Bytes   Expiration   Time Partitioning   Clustered Fields   Labels  
 ----------------- ---------------------------------- ------------ ------------- ------------ ------------------- ------------------ -------- 
  07 Jun 17:41:53   |- user_id: integer                110000       13657486                                                                  
                    |- start_version: string                                                                                                  
                    |- os_version: string                                                                                                     
                    |- client_event_time: timestamp                                                                                           
                    |- language: string                                                     